# Capstone Final Project - The Battle of Neighborhoods

You are going to move to Toronto, but you don't know which neighbourhood to choose.

Your priority is the quality of the schools around the neighbourhood that you will live.

In this project, I segmented and clustered their neighborhoods of Toronto.
I used the file https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M as the source, and http://cocl.us/Geospatial_data for the 
coordinates of the neighbourhoods.

Then I used www.foursquare.com to find the schools around each neighbourhood.

I used the file https://github.com/akinciak/Coursera_Capstone/blob/master/torontoschools.csv to get the scores of the schools.
    
Finally, I found out the best neighbourds to live according to their education quality.

Import Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Read the table

In [2]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki,'xml')
table = soup.find('table',{'class':'wikitable sortable'})
rows = table.find_all('tr')


Extract raw tables

In [3]:
frame = []
for row in rows:
    frame.append([t.text.strip() for t in row.find_all('td')])


Create a dataframe

In [4]:
df = pd.DataFrame(frame, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['Borough'].isnull()]  

Clean not assigned rows of column Borough

In [5]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True) 
new_df = df.reset_index()

Group by postal code

In [6]:
grouped_df= new_df.groupby('PostalCode').agg(lambda x: ','.join(x))

Fix the repeating Postal Codes

In [7]:
grouped_df.loc[grouped_df['Neighbourhood']=="Not assigned",'Neighbourhood']=grouped_df.loc[grouped_df['Neighbourhood']=="Not assigned",'Borough']
new_grouped_df = grouped_df.reset_index()
new_grouped_df['Borough']= new_grouped_df['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")


In [56]:
new_grouped_df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


Import CSV file

In [10]:
latlng_df = pd.read_csv('http://cocl.us/Geospatial_data')
latlng_df.columns = ['PostalCode', 'Latitude', 'Longitude']

Inner join with previous data frame

In [43]:
joined_df = pd.merge(new_grouped_df, latlng_df, on=['PostalCode'], how='inner')
joined_df.head(20)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


I will take the neighbourhoods of Toronto only.

In [45]:
toronto_df= joined_df[joined_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,EastToronto,The Beaches,43.676357,-79.293031
1,M4K,EastToronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,EastToronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,EastToronto,Studio District,43.659526,-79.340923
4,M4N,CentralToronto,Lawrence Park,43.728020,-79.388790
5,M4P,CentralToronto,Davisville North,43.712751,-79.390197
6,M4R,CentralToronto,North Toronto West,43.715383,-79.405678
7,M4S,CentralToronto,Davisville,43.704324,-79.388790
8,M4T,CentralToronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,CentralToronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


I will import all the schools around each neighbourhood.

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200207'
LIMIT = 30
radius = 1000
search_query = 'school'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [18]:
def call_schools (latitude, longitude):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']
        
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    return (dataframe_filtered)

In [50]:
joined_all_schools = pd.DataFrame(columns=['neighborhood', 'name', 'categories', 'lat', 'lng'])
for ind in toronto_df.index: 
    all_schools = call_schools(toronto_df['Latitude'][ind], toronto_df['Longitude'][ind])
    all_schools_filtered =  all_schools[['name', 'categories', 'lat', 'lng']] 
    all_schools_filtered['neighborhood'] = toronto_df['Neighbourhood'][ind]
    joined_all_schools = joined_all_schools.append(all_schools_filtered, ignore_index=True, sort=False)
joined_all_schools.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,neighborhood,name,categories,lat,lng
0,The Beaches,St.John Catholic School,School,43.680676,-79.294542
1,The Beaches,Beach Swim School,Swim School,43.682231,-79.289350
2,The Beaches,Balmy Beach School,School,43.676199,-79.290134
3,The Beaches,St. Denis Catholic School,School,43.672881,-79.290056
4,The Beaches,St. John's School Playground,Playground,43.680739,-79.296430


Remove all unnecessary columns

In [51]:
neighbourhood_schools = joined_all_schools[['neighborhood','name']]
neighbourhood_schools.head()

,neighborhood,name
0,The Beaches,St.John Catholic School
1,The Beaches,Beach Swim School
2,The Beaches,Balmy Beach School
3,The Beaches,St. Denis Catholic School
4,The Beaches,St. John's School Playground


Import the scores of the schools in Toronto.

In [52]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_828064b985ba435388baa3cf3d77c498 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_828064b985ba435388baa3cf3d77c498.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-qs1ohumzxen0f7',Key='torontoschools.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head(10)


,name,score
0,Alexander Muir Elementary School,7.5
1,Annette Street Public School,5.6
2,Annette St School,5.6
3,Balmy Beach School,7.6
4,Bedford Park Public School,8.6
5,Bowmore Public School,5.4
6,Brown Jr Public School,7.5
7,bruce junior public school,6.2
8,Church Street Public School,4.7
9,Church Street Junior Public School,4.7


Add the scores column to main dataframe.

In [53]:
neighbourhood_schools
df_data_0
toronto_schools_scores = pd.merge(neighbourhood_schools, df_data_0, on="name").sort_values('neighborhood').reset_index(drop = True)
toronto_schools_scores.head(10)

,neighborhood,name,score
0,"Adelaide,King,Richmond",Ogden Junior Public School,6.2
1,"Adelaide,King,Richmond",St Michaels Choir School,9.6
2,Berczy Park,Marketlane School,5.4
3,"Brockton,Exhibition Place,Parkdale Village",Parkdale Public School,2.8
4,"Brockton,Exhibition Place,Parkdale Village",Queen Victoria Public School,5.4
5,"Brockton,Exhibition Place,Parkdale Village",Alexander Muir Elementary School,7.5
6,"Brockton,Exhibition Place,Parkdale Village",Shirley Street Public School,6.3
7,Business Reply Mail Processing Centre 969 Eastern,St. Joseph's Catholic School,7.7
8,Business Reply Mail Processing Centre 969 Eastern,Leslieville Junior Public School,5.1
9,Business Reply Mail Processing Centre 969 Eastern,bruce junior public school,6.2


Take the average scores of the schools around each neighbourhood.

In [54]:
mean_of_neighbourhood_schools = toronto_schools_scores.groupby('neighborhood').mean().reset_index()
mean_of_neighbourhood_schools_sorted = mean_of_neighbourhood_schools.sort_values('score', ascending = False).reset_index(drop = True)

List best 10 neigbourhoods according to education quality.

In [55]:
mean_of_neighbourhood_schools_sorted.head(10)

,neighborhood,score
0,St. James Town,9.600000
1,Rosedale,8.925000
2,"Moore Park,Summerhill East",8.850000
3,Lawrence Park,8.600000
4,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",8.233333
5,Davisville,7.950000
6,"The Danforth West,Riverdale",7.925000
7,"Commerce Court,Victoria Hotel",7.900000
8,"First Canadian Place,Underground city",7.900000
9,"Design Exchange,Toronto Dominion Centre",7.900000
